In [2]:
!pip install selenium
!pip install undetected_chromedriver

  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.7
    Uninstalling urllib3-1.26.7:
      Successfully uninstalled urllib3-1.26.7


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

conda 4.10.3 requires ruamel_yaml_conda>=0.11.14, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
selenium 4.0.0b4 requires urllib3[secure]~=1.26, but you'll have urllib3 1.25.11 which is incompatible.


In [66]:
import time 
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

driver = uc.Chrome(executable_path=r'F:/chromedriver_win32/chromedriver.exe')

In [68]:
sources = []
destinations = []
print("Please enter -1 when done.")
print("-"*10)
while True:
    sources.append(input("From which city?\n"))
    if "-1" in sources: 
        sources.pop(-1)
        break
    destinations.append(input("Where to?\n"))
    if "-1" in destinations: 
        sources.pop(-1)
        destinations.pop(-1)
        break
    print("-"*10)

print("\nRoutes:")
for i in range(len(sources)):
    print(f"{sources[i]} => {destinations[i]}")

Please enter -1 when done.
----------
From which city?
BLR
Where to?
DEL
----------
From which city?
-1

Routes:
BLR => DEL


In [69]:
trip =input('Enter Trip OneWay(O) or RoundTrip(R) in Capital Letter\n')[0]
if trip=='O':
    print('OneWay')
elif trip=='R':
    print('RoundTrip')
else:
    trip =input('Enter Trip OneWay(O) or RoundTrip(R) in Capital Letter\n')[0]

Enter Trip OneWay(O) or RoundTrip(R) in Capital Letter
O
OneWay


In [71]:
from datetime import datetime
start_date = np.datetime64(input('Start Date, Please use YYYY-MM-DD format only '))
start_date1=str(start_date)
start_year, start_month, start_day = start_date1.split('-')
print(start_day)

end_date = np.datetime64(input('End Date, Please use YYYY-MM-DD format only '))
end_date1 = str(end_date)
end_year, end_month, end_day =end_date1.split('-')
print(end_day)

days = end_date - start_date
num_days = days.item().days

Start Date, Please use YYYY-MM-DD format only 2021-11-05
05
End Date, Please use YYYY-MM-DD format only 2021-11-06
06


In [72]:
class_type =input('Enter class type (Economy/Business/Special)\n')
if class_type=='Economy'or class_type=='Business'or class_type=='Special':
    print(class_type)
else:
    class_type =input('Enter class type\n')

Enter class type (Economy/Business/Special)
Special
Special


In [79]:
def get_airlines(soup):
    airline = []
    airlines = soup.find_all('p',class_='normal fs-10 fs-10 abs font-lightestgrey no-wrap mt-2 fl-no',text=True)
    for i in airlines:
        airline.append(i.text)
    return airline
    
def get_total_stops(soup):
    stops_list = []
    stops = soup.find_all('span',class_='dotted-borderbtm')

    for i in stops:
        for j in i.find_all('span',class_='dotted-borderbtm'):
               stops_list.append(j.text)
    return stops_list

def get_price(soup):
    prices = []
    price = soup.find_all('div',class_='pr i-b mr-2')

    for i in price:
        for j in i.find_all('span', class_='pr i-b mr-2'):
            prices.append(j.text)
    return prices

def get_duration(soup):
    duration_list = []
    duration = soup.find_all('div' , class_='fs-12 bold du mb-2')
    for i in duration:
        for j in i.find_all('div',class_='fs-12 bold du mb-2'):
            duration_list.append(j.text)
    return duration_list

def arr_dep_time(soup):
    depart = []
    depart = soup.find_all('div',class_='bold fs-16 mb-2',text=True)
    for i in depart:
        depart.append(i.text)
    return depart

def arr_dep_day(soup):
    day = []
    day = soup.find_all('div',class_='bold fs-12 mb-5',text=True)
    for i in day:
        day.append(i.text)
    return day



In [80]:
for i in range(len(sources)):
    column_names = ["Airline", "Source", "Destination","Duration" ,"Depart","Arr","Total stops", "Price","Date"]
    df = pd.DataFrame(columns = column_names)
    for j in tqdm(range(num_days+1)):
   
        if j % 10 == 0:
            driver.quit()
           
            driver = uc.Chrome(executable_path=r'F:/chromedriver_win32/chromedriver.exe')
        url=f"https://flight.yatra.com/air-search-ui/dom2/trigger?type=" + trip + "&viewName=normal&flexi=0&noOfSegments=2&origin=" + sources[i] + "&originCountry=IN&destination=" + destinations[i] + "&destinationCountry=IN&flight_depart_date=" + start_day +"%2F" + start_month + "%2F" + start_year + "&arrivalDate=" + end_day + "%2F" + end_month + "%2F" + end_year +  "&ADT=1&CHD=0&INF=0&class=" + class_type + "&source=fresco-home&unqvaldesktop=1423737925126"

        driver.get(url)
        sleep(15)

        driver.execute_script("window.scrollBy(0,100)")
        time.sleep(1)
 
        try:
            flight_detail_link = driver.find_elements(By.XPATH,'//div[@class = "link-color fs-12 cursor-pointer"]')
        except:
            
            input("Please solve the captcha then enter anything here to resume scraping.")
            
        while True:
            try:
                flight_detail_link.click()
            except:
                break
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        airlines = get_airlines(soup)
        total_stops = get_total_stops(soup)
        prices = get_price(soup)
        duration = get_duration(soup)
        df = df.append(pd.DataFrame({
            'Airline': airlines,
            'Duration': duration,
            'Total stops' : total_stops,
            'Price' : prices,
            'Date' : start_date+j
                                    }))
        
        df['Source'] = sources[i]
        df['Destination'] = destinations[i]
        df = df.replace('\n','', regex=True)
        df = df.reset_index(drop = True)
  
        df.to_csv(f'{sources[i]}_{destinations[i]}.csv',index=False)
        print(f"Succesfully saved {sources[i]} => {destinations[i]} route as {sources[i]}_{destinations[i]}.csv ")
    
driver.quit()

#df

 50%|█████     | 1/2 [00:37<00:37, 37.53s/it]

Succesfully saved BLR => DEL route as BLR_DEL.csv 


100%|██████████| 2/2 [00:54<00:00, 27.09s/it]

Succesfully saved BLR => DEL route as BLR_DEL.csv 
